# Panel Loading

Notebook to load panel data into the databse

## Imports

In [1]:
import psycopg2
from tabulate import tabulate
#import a folder in the parent directory
import sys
sys.path.append('../')
import dbtools.dbtools as qrs
import dbtools.load as load

## Connection

In [2]:
try:
    # Connect to the PostgreSQL database
    conn = qrs.connect()
    print("Connected to the database")

except (Exception, psycopg2.DatabaseError) as error:
    print(error)

Connected to the database


## Material id search

A panel is made of a material, so the panel table is linked to the material table.

Here we have to check the material table and to set the material of the panel.

In [3]:
parent_table_name = 'materials'

parent_id_column = parent_table_name[:-1] + '_id'

parent_data = qrs.get_data(parent_table_name)

parent_data

,id_material,name_material
0,1,Juan Ignacio
1,15,Prueba


In [4]:
#we set the id
material_id = 15

table_name = 'panels'

In [5]:
load.load_panel(conn,'prueba',material_id,[1,2,3],False,'Descripcion')

Panel 'prueba' loaded with ID: 3


## Make sure insert was correct

In [6]:
data = qrs.get_data_metadata(table_name)

data

,id_panel,name_panel,material_id_panel,description_panel,dim_panel,edges_cutted_panel
0,1,Juan Ignacio,1,None,NaN,NaN
1,3,prueba,15,Descripcion,"[1, 2, 3] mm",False bool


In [7]:
conn.close()